In [1]:
import warnings
warnings.filterwarnings('ignore')

from io import StringIO 
import sys

import json
import pandas as pd
import os
from vega import VegaLite
from ipywidgets import widgets
import numpy as np

from falx.chart import *
from eval_interface import FalxEvalInterface

DATA_DIR = os.path.join("..", "benchmarks")

np.random.seed(2019)

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        self._stderr = sys.stderr
        sys.stdout = self._stringio = StringIO()
        sys.stderr = self._stringio
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout
        sys.stderr = self._stderr
        
def render_vegalite(vis):
    # Render a visualization using vegalite
    VegaLite(vis.to_vl_obj()).display()
    
def print_log(output):
    print_flag = False
    for l in output:
        if print_flag:
            print(l)
        if "[info] #Candidates before getting the correct solution:" in l:
            print(l)
        if "====>" in l:
            print_flag = True


def run_synthesis(fname, num_samples):
    with open(os.path.join(DATA_DIR, fname), "r") as f:
        data = json.load(f)

    print("# run synthesize {}".format(fname))

    input_data = data["input_data"]
    vis = VisDesign.load_from_vegalite(data["vl_spec"], data["output_data"])
    trace = vis.eval()    
    result = FalxEvalInterface.synthesize(inputs=[input_data], full_trace=trace, num_samples=num_samples)
    print("====>")
    for p, vis in result:
        print("# table_prog:")
        print("  {}".format(p))
        print("# vis_spec:")
        vl_obj = vis.to_vl_obj()
        data = vl_obj.pop("data")["values"]
        print("    {}".format(vl_obj))
            
def run_wrapper(fname, num_samples=4):
    print("\n====> {}".format(fname))
    with Capturing() as output:
        run_synthesis(fname, num_samples)
    print_log(output)

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
#simple cases
%time run_wrapper('001.json')
%time run_wrapper('002.json')
%time run_wrapper('005.json')
%time run_wrapper('006.json')
%time run_wrapper('007.json')
%time run_wrapper('009.json')
%time run_wrapper('010.json')
%time run_wrapper('011.json')
%time run_wrapper('012.json')
%time run_wrapper('022.json')
%time run_wrapper('024.json')
%time run_wrapper('025.json')
%time run_wrapper('027.json')
%time run_wrapper('031.json')
%time run_wrapper('032.json')
%time run_wrapper('033.json')
%time run_wrapper('035.json')
%time run_wrapper('038.json')
%time run_wrapper('041.json')
%time run_wrapper('042.json')
%time run_wrapper('045.json')
%time run_wrapper('046.json')
%time run_wrapper('050.json')
%time run_wrapper('052.json')
%time run_wrapper('053.json')
%time run_wrapper('054.json')
%time run_wrapper('056.json')
%time run_wrapper('058.json')
%time run_wrapper('test_1.json')
%time run_wrapper('test_4.json')
%time run_wrapper('test_6.json')
%time run_wrapper('test_7.json')
%time run_wrapper('test_10.json')
%time run_wrapper('test_14.json')
%time run_wrapper('test_15.json')
%time run_wrapper('test_17.json')
%time run_wrapper('test_21.json')
%time run_wrapper('test_22.json')

In [ ]:
# slow case
%time run_wrapper('003.json')
%time run_wrapper('004.json') # ~1min
%time run_wrapper('013.json') # ~30s
%time run_wrapper('014.json') # ~2min
%time run_wrapper('015.json') # ~2min
%time run_wrapper('016.json') # ~30s
%time run_wrapper('017.json') # ~30s
%time run_wrapper('020.json') # ~1min
%time run_wrapper('023.json') # ~30s
%time run_wrapper('029.json') # ~2min
%time run_wrapper('030.json') # ~2min
%time run_wrapper('034.json') # ~1min
%time run_wrapper('040.json')
%time run_wrapper('043.json') # ~2min
%time run_wrapper('047.json')
%time run_wrapper('051.json') # ~1min
%time run_wrapper('055.json') # ~30s
%time run_wrapper('test_9.json') # ~2min

In [ ]:
#very slow cases
%time run_wrapper('008.json') # can be solved with: gatherNeg, spread
%time run_wrapper('021.json') # solved with: mutate, gather
%time run_wrapper('026.json') # solved with: separate, gbsum
%time run_wrapper('037.json') # solved with: gather, mutate
%time run_wrapper('044.json') # solved with: gather, gbsum
%time run_wrapper('test_2.json') # can be solved with: gatherNeg, separate
%time run_wrapper('test_5.json') # can be solved with: gatherNeg, gb_sum
%time run_wrapper('test_8.json') # can be solved with: mutate, unite
%time run_wrapper('test_11.json') # can be solved with: gather + cumsum
%time run_wrapper('test_12.json') # can be solved with: gather + gbsum
%time run_wrapper('test_13.json') # can be solved with: gather + unite
%time run_wrapper('test_16.json') # can be solved with: mutate + gather
%time run_wrapper('test_19.json') # can be solved with: mutate + gather
%time run_wrapper('test_20.json') # can be solved with: mutate / gather
%time run_wrapper('test_23.json') # can be solved with: gbsum + gatherNeg#

In [ ]:
# 3 step, we should try
#%time run_wrapper('018.json') # cannot solve, 3 steps: gather, pivot, gather
#%time run_wrapper('039.json') # cannot solve 3 steps: gather, separate, spread
#%time run_wrapper('048.json') # cannot solve 3 steps: gbsum, gather, mutate
#%time run_wrapper('049.json') # cannot solve, 2 steps require 3 way split
#%time run_wrapper('059.json') # cannot solve, 3 steps: gather, split, spread

In [ ]:
# cannot solve, give up
#%time run_wrapper('019.json') # cannot solve, cut
#%time run_wrapper('028.json') # cannot solve, cut
#%time run_wrapper('036.json') # cannot solve, require shift()
#%time run_wrapper('057.json') # cannot solve, cumcount
#%time run_wrapper('060.json') # cannot solve, cumcount
#%time run_wrapper('test_3.json') # cannot solve, require shift()
#%time run_wrapper('test_18.json') # needs bigger gather (size 4), may give up